In [ ]:
from functools import reduce
from sklearn.model_selection import train_test_split

import torch

In [ ]:
X = []
reduce(lambda x, y: X.extend(list(map(lambda z: [y, z+1], range(10)))), range(11))

print(X)

[[1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [3, 10], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [4, 10], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9], [5, 10], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6], [6, 7], [6, 8], [6, 9], [6, 10], [7, 1], [7, 2], [7, 3], [7, 4], [7, 5], [7, 6], [7, 7], [7, 8], [7, 9], [7, 10], [8, 1], [8, 2], [8, 3], [8, 4], [8, 5], [8, 6], [8, 7], [8, 8], [8, 9], [8, 10], [9, 1], [9, 2], [9, 3], [9, 4], [9, 5], [9, 6], [9, 7], [9, 8], [9, 9], [9, 10], [10, 1], [10, 2], [10, 3], [10, 4], [10, 5], [10, 6], [10, 7], [10, 8], [10, 9], [10, 10]]


In [ ]:
y = list(map(lambda x: [x[0]*x[1]/100], X))
print(y)

[[0.01], [0.02], [0.03], [0.04], [0.05], [0.06], [0.07], [0.08], [0.09], [0.1], [0.02], [0.04], [0.06], [0.08], [0.1], [0.12], [0.14], [0.16], [0.18], [0.2], [0.03], [0.06], [0.09], [0.12], [0.15], [0.18], [0.21], [0.24], [0.27], [0.3], [0.04], [0.08], [0.12], [0.16], [0.2], [0.24], [0.28], [0.32], [0.36], [0.4], [0.05], [0.1], [0.15], [0.2], [0.25], [0.3], [0.35], [0.4], [0.45], [0.5], [0.06], [0.12], [0.18], [0.24], [0.3], [0.36], [0.42], [0.48], [0.54], [0.6], [0.07], [0.14], [0.21], [0.28], [0.35], [0.42], [0.49], [0.56], [0.63], [0.7], [0.08], [0.16], [0.24], [0.32], [0.4], [0.48], [0.56], [0.64], [0.72], [0.8], [0.09], [0.18], [0.27], [0.36], [0.45], [0.54], [0.63], [0.72], [0.81], [0.9], [0.1], [0.2], [0.3], [0.4], [0.5], [0.6], [0.7], [0.8], [0.9], [1.0]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, train_size=66)

In [ ]:
X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.float32)

In [ ]:
class Network(torch.nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(Network, self).__init__()

        self.layer1 = torch.nn.Linear(input_size, hidden_size)
        # print(self.layer1.weight)
        # print(type(self.layer1.weight))
        self.layer1.weight.data = torch.tensor(list(map(lambda x: list(map(lambda y: 0.5, range(input_size))), range(hidden_size))))
        # print(self.layer1.weight)
        self.layer2 = torch.nn.Linear(hidden_size, hidden_size)
        self.layer2.weight.data = torch.tensor(list(map(lambda x: list(map(lambda y: 0.5, range(hidden_size))), range(hidden_size))))
        self.layer3 = torch.nn.Linear(hidden_size, output_size)
        self.layer3.weight.data = torch.tensor(list(map(lambda x: list(map(lambda y: 0.5, range(hidden_size))), range(output_size))))

        self.weight_history = {}
        self.prev_weight = [self.layer1.weight.data.clone().detach(), self.layer2.weight.data.clone().detach(), self.layer3.weight.data.clone().detach()]


    def forward(self, x):

        x1 = torch.nn.functional.relu(self.layer1(x))
        x2 = torch.nn.functional.relu(self.layer2(x1))
        x3 = self.layer3(x2)

        return x3

    def fix_weigt_changes(self):

      idx = max(self.weight_history.keys(), default=0) + 1
      self.weight_history[idx] = [torch.abs(self.prev_weight[0] - self.layer1.weight.data).clone().detach(),
                                  torch.abs(self.prev_weight[1] - self.layer2.weight.data).clone().detach(),
                                  torch.abs(self.prev_weight[2] - self.layer3.weight.data).clone().detach()]
      self.prev_weight = [self.layer1.weight.data.clone().detach(), self.layer2.weight.data.clone().detach(), self.layer3.weight.data.clone().detach()]


In [ ]:
def print_weights(m):
    if 'weight' in dir(m):
        print(m.weight)

In [ ]:
input_size = 2
hidden_size = 3
output_size = 1

In [ ]:
model = Network(input_size, hidden_size, output_size)

In [ ]:
# model.apply(print_weights)

In [ ]:
loss_fn = torch.nn.MSELoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00005)

In [ ]:
def accuracy(y_true, y_pred):

    t_compare = torch.eq(y_true, y_pred)
    result = torch.count_nonzero(t_compare)/len(y_true)*100

    return result

In [ ]:
def train(number_of_epochs):

    for epoch in range(number_of_epochs):

        model.train()

        y_pred = model(X_train)
        loss = loss_fn(y_train, y_pred)
        acc = accuracy(y_train, y_pred)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.fix_weigt_changes()

        if epoch % 100 == 0:
            model.eval()

            y_pred = model(X_test)
            test_loss = loss_fn(y_test, y_pred)
            test_acc = accuracy(y_test, y_pred)

            print(f'Epoch: {epoch}, loss: {loss: .5f}, acc: {acc: .5f}%, test loss: {test_loss: .5f}, test acc: {test_acc: .5f}%')

    y_pred = model(X_test)
    test_loss = loss_fn(y_test, y_pred)
    test_acc = accuracy(y_test, y_pred)

    print(f'Epoch: {epoch}, loss: {loss: .5f}, acc: {acc: .5f}%, test loss: {test_loss: .5f}, test acc: {test_acc: .5f}%')

In [ ]:
train(1000)

Epoch: 0, loss:  202.12016, acc:  0.00000%, test loss:  166.05162, test acc:  0.00000%
Epoch: 100, loss:  63.62353, acc:  0.00000%, test loss:  52.54877, test acc:  0.00000%
Epoch: 200, loss:  26.66020, acc:  0.00000%, test loss:  22.13005, test acc:  0.00000%
Epoch: 300, loss:  13.37070, acc:  0.00000%, test loss:  11.14079, test acc:  0.00000%
Epoch: 400, loss:  7.48974, acc:  0.00000%, test loss:  6.25936, test acc:  0.00000%
Epoch: 500, loss:  4.50907, acc:  0.00000%, test loss:  3.77770, test acc:  0.00000%
Epoch: 600, loss:  2.85074, acc:  0.00000%, test loss:  2.39354, test acc:  0.00000%
Epoch: 700, loss:  1.86469, acc:  0.00000%, test loss:  1.56888, test acc:  0.00000%
Epoch: 800, loss:  1.24923, acc:  0.00000%, test loss:  1.05341, test acc:  0.00000%
Epoch: 900, loss:  0.85105, acc:  0.00000%, test loss:  0.71957, test acc:  0.00000%
Epoch: 999, loss:  0.58869, acc:  0.00000%, test loss:  0.49947, test acc:  0.00000%


In [ ]:
model.weight_history[1]

[tensor([[0.0010, 0.0010],
         [0.0010, 0.0010],
         [0.0010, 0.0010]]),
 tensor([[0.0010, 0.0010, 0.0010],
         [0.0010, 0.0010, 0.0010],
         [0.0010, 0.0010, 0.0010]]),
 tensor([[0.0010, 0.0010, 0.0010]])]

In [ ]:
model.weight_history[len(model.weight_history)]

[tensor([[6.5997e-05, 6.5953e-05],
         [6.9246e-05, 6.9201e-05],
         [6.6787e-05, 6.6757e-05]]),
 tensor([[6.4194e-05, 5.5060e-05, 6.1959e-05],
         [6.7085e-05, 5.7563e-05, 6.4760e-05],
         [6.7636e-05, 5.8040e-05, 6.5282e-05]]),
 tensor([[6.3628e-05, 5.3808e-05, 5.1945e-05]])]

In [ ]:
x = torch.tensor([6, 6], dtype=torch.float32)
y = model(x)
print(y)
print(torch.round(y*100))

tensor([-0.2849], grad_fn=<AddBackward0>)
tensor([-28.], grad_fn=<RoundBackward0>)


In [ ]:
res = []
hist_idx = 1000
for i in range(len(model.weight_history[hist_idx])):
    # print(f'i: {i} len: {len(model.weight_history[1][i])}')
    for j in range(len(model.weight_history[hist_idx][i])):
        # print(f'j: {j} len: {len(model.weight_history[1][i][j])}')
        # print(model.weight_history[1][i][j])
        for k in model.weight_history[hist_idx][i][j]:
            # print(f'k: {k}') #' len: {len(model.weight_history[1][i][j][k])}')
            # print(type(k))
            res.extend([k.item()])

print(res)

[6.599724292755127e-05, 6.595253944396973e-05, 6.924569606781006e-05, 6.920099258422852e-05, 6.67870044708252e-05, 6.67572021484375e-05, 6.41942024230957e-05, 5.505979061126709e-05, 6.195902824401855e-05, 6.708502769470215e-05, 5.7563185691833496e-05, 6.476044654846191e-05, 6.763637065887451e-05, 5.804002285003662e-05, 6.528198719024658e-05, 6.362795829772949e-05, 5.380809307098389e-05, 5.194544792175293e-05]


In [ ]:
len(model.weight_history)

1000

In [ ]:
res = []
# for i in range(len(model.weight_history[1])):
#     # print(f'i: {i} len: {len(model.weight_history[1][i])}')
#     for j in range(len(model.weight_history[1][i])):
#         # print(f'j: {j} len: {len(model.weight_history[1][i][j])}')
#         # print(model.weight_history[1][i][j])
#         for k in model.weight_history[1][i][j]:
#             # print(f'k: {k}') #' len: {len(model.weight_history[1][i][j][k])}')
#             # print(type(k))
#             res.extend([k.item()])

print(reduce(lambda j0, j: res.extend(list(map(lambda k: k.item(), model.weight_history[1000][0][j-1]))), range(len(model.weight_history[1000][0])+1)))

print(res)

None
[6.599724292755127e-05, 6.595253944396973e-05, 6.924569606781006e-05, 6.920099258422852e-05, 6.67870044708252e-05, 6.67572021484375e-05]
